In [1]:
from alphafed import mock_context
from alphafed.auto_ml import from_pretrained

auto_model = from_pretrained(resource_dir='auto_model_fed_avg')

2023-02-08 02:38:12,712|INFO|pretrained|from_pretrained|34:
Loading resouce from: `auto_model_fed_avg`.
2023-02-08 02:38:12,713|DEBUG|pretrained|_load_model_obj|140:
config.entry_file='auto_fed_avg.py'
2023-02-08 02:38:12,714|DEBUG|pretrained|_load_model_obj|141:
model_file='auto_model_fed_avg/auto_fed_avg.py'
2023-02-08 02:38:12,789|DEBUG|pretrained|_load_model_obj|143:
module=<module 'auto_fed_avg' from '/app/db/notebook_dir/user_07ec421f72/docs/tutorial/res/auto_model_fed_avg/auto_fed_avg.py'>
2023-02-08 02:38:12,790|DEBUG|pretrained|_load_model_obj|145:
model_class=<class 'auto_fed_avg.AutoResNetFedAvg'>
2023-02-08 02:38:12,791|INFO|pretrained|_load_model_obj|147:
Loading pretrained model complete.


测试加载训练数据。

In [2]:
is_succ, help_text = auto_model.init_dataset(dataset_dir='data/HAM10000')
print(f'数据是否加载成功: {is_succ}')
print(f'提示信息: {help_text}')
if is_succ:
    print(f'包含训练集样本: {len(auto_model.training_loader.dataset)}')
    print(f'包含验证集样本: {len(auto_model.validation_loader.dataset)}')
    print(f'包含测试集样本: {len(auto_model.testing_loader.dataset)}')

数据是否加载成功: True
提示信息: Initializing dataset complete.
包含训练集样本: 7012
包含验证集样本: 1000
包含测试集样本: 2003


微调训练之前的测试。

In [3]:
avg_loss, correct_rate = auto_model.run_test()
print(f'平均损失为: {avg_loss}')
print(f'准确率为: {correct_rate:.2f}')

2023-02-08 02:38:20,226|INFO|auto_fed_avg|push_log|488:
Begin testing of epoch 0.
2023-02-08 02:39:14,198|INFO|auto_fed_avg|run_test|412:
Testing Average Loss: 2.4185
2023-02-08 02:39:14,199|INFO|auto_fed_avg|run_test|413:
Testing Correct Rate: 5.59


平均损失为: 2.4184583753451547
准确率为: 5.59


由于本地数据集类别与模型预训练使用的类别完全不同，所以初始准确率很低，只有 5.59%。现在正式开始微调。

In [4]:
import logging
from alphafed import logger

# 关闭 DEBUG 日志，如果需要更详细的调试日志，可以将这一行注释掉
logger.setLevel(logging.INFO)

task_id = '11c12dc5-0473-4932-930e-ad56c69c5ea1'  # 必须与聚合方配置相同
aggregator_id = 'd5f978fa-84f5-4724-b4f5-8abb317be4e2'  # 必须与聚合方配置相同
col_id_1 = '4d43ea09-aad6-4beb-bc23-105e90ad5567'   # 必须与聚合方配置相同
col_id_2 = 'ff2ce0a2-6983-45d6-8512-151e71710928'  # 必须与聚合方配置相同
with mock_context(id=aggregator_id, nodes=[aggregator_id, col_id_1, col_id_2]):  # 在模拟调试环境中运行
    auto_model.fine_tune(id=aggregator_id,
                         task_id=task_id,
                         dataset_dir='data/HAM10000',
                         is_initiator=True)

2023-02-08 02:39:25,357|INFO|scheduler|push_log|118:
Begin to validate local context.
2023-02-08 02:39:25,358|INFO|scheduler|push_log|118:
There are 2003 samples for testing.
2023-02-08 02:39:25,359|INFO|scheduler|push_log|118:
Local context is ready.
2023-02-08 02:39:25,360|INFO|scheduler|push_log|118:
Node d5f978fa-84f5-4724-b4f5-8abb317be4e2 is up.
2023-02-08 02:39:25,361|INFO|scheduler|push_log|118:
Waiting for participants taking part in ...
2023-02-08 02:39:37,391|INFO|scheduler|push_log|118:
Welcome a new participant ID: 4d43ea09-aad6-4beb-bc23-105e90ad5567.
2023-02-08 02:39:37,392|INFO|scheduler|push_log|118:
There are 2 participants now.
2023-02-08 02:39:59,449|INFO|scheduler|push_log|118:
Welcome a new participant ID: ff2ce0a2-6983-45d6-8512-151e71710928.
2023-02-08 02:39:59,450|INFO|scheduler|push_log|118:
There are 3 participants now.
2023-02-08 02:39:59,460|INFO|scheduler|push_log|118:
Synchronizing round state ...
2023-02-08 02:39:59,461|INFO|scheduler|push_log|118:
Initi

完成微调后测试推理。

In [5]:
image_file = 'data/HAM10000/ISIC_0033103.jpg'  # 随便选一张图片
predict = auto_model(image_file)
print(f'推理结果为: {predict}')

推理结果为: MEL
